In [1]:
import getpass
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from hdbscan import HDBSCAN
import matplotlib.pyplot as plt
import seaborn as sns

## Loading data

In [5]:
path2SP = f"/Users/{getpass.getuser()}/OneDrive - World Justice Project/EU Subnational/EU-S Data"
eugpp = pd.read_stata(
        f"{path2SP}/eu-gpp/1. Data/3. Merge/EU_GPP_2024.dta", 
        convert_categoricals = False
    )
eugpp = (
    eugpp
    .loc[(~eugpp["country_name_ltn"].isin(["Luxembourg"])) & (eugpp["gend"] < 3)]
)
nlpca_results = (
    pd.read_csv("../../../data/nlpca_results.csv")
    .set_index("country_year_id")
    .query("group == 'AA'")
)
features = (
    nlpca_results.copy()
    .drop(columns = ["group"])
)
features_reduced = (
    pd.read_csv("../../../data/umap_reductions/umap_AA_results.csv")
    .set_index("country_year_id")
)

In [6]:
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)

In [ ]:
# Exploring the K-nearest neighbors distance plot
min_samples_0 = 2*len(features.columns)
k = min_samples_0-1
neighbors = NearestNeighbors(n_neighbors=k)
neighbors_fit = neighbors.fit(features_normalized)
distances, indices = neighbors_fit.kneighbors(features_normalized)

distances = np.sort(distances[:, -1])
plt.figure(figsize=(8, 5))
plt.plot(distances)
plt.hlines(y=[0.25, 0.30, 0.35], xmin = 0, xmax = len(distances), color='r', linestyle='-')
plt.title("k-Nearest Neighbors Distance Plot")
plt.xlabel("Points (sorted by distance)")
plt.ylabel("Distance to 4th Nearest Neighbor")
plt.show()

In [91]:
def get_HDBSCAN_tunning_results (features, mcs, mss, eps):

    if eps == 0.0:
        hdbscan = HDBSCAN(
            min_cluster_size  = mcs,
            min_samples       = mss,
            metric            = "euclidean",
            gen_min_span_tree = True
        )
    else:
        hdbscan = HDBSCAN(
            min_cluster_size  = mcs,
            min_samples       = mss, 
            metric            = "euclidean",
            gen_min_span_tree = True,
            cluster_selection_epsilon = eps,
        )
    
    labels     = hdbscan.fit(features_normalized).labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise    = list(labels).count(-1)
    score      = hdbscan.relative_validity_
    cluster_persistence = hdbscan.cluster_persistence_
    
    results = {
        "min_cluster_size" : mcs,
        "min_samples"      : mss,
        "epsilon"          : eps,
        "n_clusters"       : len(set(hdbscan.labels_)),
        "n_noise"          : list(hdbscan.labels_).count(-1),
        "relative_validity": score,
        "cpersistance"     : np.round(cluster_persistence, 3)
    }
    
    return(results)

In [92]:
min_cluster_size = [250, 500, 750]
min_samples_list = [None, 50, 250]
eps_list = [0.0, 0.25, 0.30, 0.35]

hyper_tunning_results = [
    get_HDBSCAN_tunning_results(features_normalized, mcs, mss, eps) 
    for mcs in min_cluster_size 
    for mss in min_samples_list 
    for eps in eps_list
]

In [93]:
pd.DataFrame(hyper_tunning_results).fillna(0)

,min_cluster_size,min_samples,epsilon,n_clusters,n_noise,relative_validity,cpersistance
0,250,0.0,0.00,1,16036,0.0,[]
1,250,0.0,0.25,1,16036,0.0,[]
2,250,0.0,0.30,1,16036,0.0,[]
3,250,0.0,0.35,1,16036,0.0,[]
4,250,50.0,0.00,1,16036,0.0,[]
5,250,50.0,0.25,1,16036,0.0,[]
6,250,50.0,0.30,1,16036,0.0,[]
7,250,50.0,0.35,1,16036,0.0,[]
8,250,250.0,0.00,1,16036,0.0,[]
9,250,250.0,0.25,1,16036,0.0,[]


In [28]:
# Final parameters
min_cluster_size = 250
min_samples = None
eps = None

# Fitting a DBSCAN model
if eps is None:
    hdbscan = HDBSCAN(
        min_cluster_size = min_cluster_size,
        min_samples = min_samples, 
        # cluster_selection_epsilon = eps,
        metric = "euclidean",
        gen_min_span_tree = True
    )
else:
    hdbscan = HDBSCAN(
        min_cluster_size = min_cluster_size,
        min_samples = min_samples, 
        cluster_selection_epsilon = eps,
        metric = "euclidean",
        gen_min_span_tree = True
    )

hdbscan.fit(features_normalized)

n_clusters = len(set(hdbscan.labels_)) - (1 if -1 in hdbscan.labels_ else 0)
n_noise = list(hdbscan.labels_).count(-1)

In [ ]:
# Saving labels
eugpp["cluster_hdbscan"] = hdbscan.labels_
nlpca_results["cluster_hdbscan"] = hdbscan.labels_
nlpca_results.cluster_hdbscan.value_counts()

## Clusters Characterization

In [ ]:
nlpca_results.groupby("cluster_gmm")[features.columns].agg("mean")

In [93]:
p1_indicators = theoretical_outline.loc[theoretical_outline["pillar"] == "Pillar_1"].target_var

In [ ]:
original_features = theoretical_outline["target_var"].drop_duplicates()
eugpp[original_features] = eugpp[original_features].replace({
    98 : np.nan,
    99 : np.nan
})
eugpp.groupby("cluster_gmm")[p1_indicators].agg("mean")